In [0]:
import numpy as np

#Initialize identifier value

m=3 # ie- values will range from 0 to 7

# here we are considering direct values without sha-1 encryption for easy verification of our work

global identifier_circle# maintained for joining two nodes

identifier_circle=[]

In [0]:
# define basic structure of a node 

class Node:
    def __init__(self,id):
      self.id=id
      self.predecessor=None
      self.finger_table=[]  #max m entries in finger table ;contains pair(start_id,succ node)
      self.keys=[]

In [0]:
#Functionality1-----------finding the successor node of an identifier   O(log n) complexity

#successor of a node

def successor(x):
  return x.finger_table[0][1]

#Find id’s successor by finding the immediate predecessor of the id

def find_predecessor(node,id):
  x=node
  #while( not((x.id < id and successor(x).id >= id) or (successor(x).id <= id and x.id < id))):
  while( not((x.id < id and successor(x).id >= id) or (successor(x).id <= x.id and (id <= successor(x).id or id > x.id)) ) ):
    x=closest_preceding_finger(x,id)
    print(x.id)
  return x

#Start with the mth finger of node n See if it comes between node n and the id, if not, check the m-1th finger until we find one wich does.
#This is the closest node preceding id among all the fingers of n

def closest_preceding_finger(node,id):
  for i in range(m-1,-1,-1):
    if((node.id<node.finger_table[i][1].id and node.finger_table[i][1].id<id) or (node.id>=id and (node.finger_table[i][1].id>node.id or node.finger_table[i][1].id<id)) ):
      return node.finger_table[i][1]
  return node
  

  # returns id of node where key with given id can be found
def find_successor(node,id):
  x=find_predecessor(node,id)
  return successor(x)
  

In [0]:
#Functionality2....... display the contents of a node in the hash table

def diplay_node_contents(id):
  
  global identifier_circle
  
  for i in range(len(identifier_circle)):
    if (identifier_circle[i][0]==id):
      node=identifier_circle[i][1]
  
  print("keys stored are.....")
  for i in range(len(node.keys)):
    print(node.keys[i])
    
  print("id is:")
  print(node.id)
  
  print("predecessor is:")
  print(node.predecessor)
  
  print("finger table is:")
    
  for i in range(m):
    print("index:"+str(i),"start"+str(node.finger_table[i][0]),node.finger_table[i][1])
  

In [0]:
#allocate keys to the appropriate node

def allocate_key(id):
  
  global identifier_circle
  flag=0
  for i in range(len(identifier_circle)):
    if(identifier_circle[i][0]>=id):
      identifier_circle[i][1].keys.append(id)
      flag=1
      break
  if(flag==0):
    identifier_circle[len(identifier_circle)-1][1].keys.append(id)
    
      

In [0]:
# Functionality3......joining of a new node into the network            O (log**2 n) Complexity

#initialize finger table of local node n

def init_finger_table(x,y):
  
  start_1=(x.id+1)%(2**m)
  second=find_successor(y,start_1)
  x.finger_table.append((start_1,second))
  x.predecessor=x.finger_table[0][1].predecessor
  x.finger_table[0][1].predecessor=x
  
  for i in range(1,m):
    start_x=(x.id+(2**i))%(2**m)
    #if start belong to [n, finger[i-1].id)
    if( (start_x>=x.id and start_x<x.finger_table[i-1][1].id) or (x.id>=x.finger_table[i-1][1].id and (start_x>=x.id or start_x<x.finger_table[i-1][1].id)) ):
      sec=x.finger_table[i-1][1]
      x.finger_table.append((start_x,sec))
      
    else:
      sec=find_successor(y,start_x)
      x.finger_table.append((start_x,sec))
      
      


# if s is the ith finger of n ,update n's finger table with s
def update_finger_table(n,s,i):
   print(str(n.id) + " " + str(s.id) + " " + str(i))
#    if(s.id>=n.id and s.id<n.finger_table[i][1].id):
   if((s.id>=n.id and s.id<n.finger_table[i][1].id) or (n.id>=n.finger_table[i][1].id and (s.id>=n.id or s.id < n.finger_table[i][1].id)) ):
    n.finger_table[i]=(n.finger_table[i][0],s)
    p=n.predecessor  #first node recedeing n
    update_finger_table(p,s,i)
  
#update all finger nodes whose finger table should refer to n
def update_others(node):
  
  for i in range(0,m):
    p=find_predecessor(node, node.id-(2**i)) # find last node p whose ith finger may be n
    update_finger_table(p,node,i)
    
  


def join_nodes(x,y):
  
  init_finger_table(x,y)
  update_others(x)
  #move keys in (predecessor,n] from successor
  

def join(id):
  
  global identifier_circle
  
  x=Node(id)  # create node with given id
  identifier_circle.append((id,x))
  sort_list=sorted(identifier_circle,key=lambda x:x[0])
  identifier_circle=sort_list
  
  length=len(identifier_circle)
  
  if(length==1):    # n is the only node in the network
    for i in range(0,m):
      start=(x.id+(2**i))%(2**m)
      x.finger_table.append((start,x))
    x.predecessor=x
    
  elif length==2:
    y=identifier_circle[0][1] #first node
    for i in range(0,m):
      start=(x.id+(2**i))%(2**m)
      x.finger_table.append((start,y)) #set every entry to first node
    x.predecessor=y
    for i in range(0,m):
      start=(y.id+(2**i))%(2**m)
      y.finger_table[i]=(start, x)  #set every entry of 1st node = second node
    y.predecessor=x
  
  else:
    y=identifier_circle[0][1]    #select random node apart form node
    join_nodes(x,y)
  

In [0]:
# Functionality 4 ----system stabilization

# set a time interval of 2 sec and perform stabilize action periodically

def stabilize(node): # periodically verify n's immediate successor and tell the successsor about n
  
  successor=node.finger_table[0][1]
  x=successor.predecessor
  if(node.id<x.id and x.id<successor.id):
    node.finger_table[0][1]=x
    notify(x,node)
    
    
def notify(n,n1): #n1 thinks it might be our predecessor
  
  if(n.predecessor==None or (n.predecessor.id <n1.id and n1.id<n.id)):
    n.predecessor=n1
    
    
    
def fix_fingers(node):  #periodically refresh finger entries
  
  for i in range(m):
    node.finger_table[i]=(node.finger_table[i][0], find_successor(node,node.finger_table[i][0]))
  
  
  
    
    
    

In [0]:
join(1)

In [0]:
allocate_key(1)

In [0]:
join(7)

In [0]:
allocate_key(7)

In [12]:
join(3)

7
1
1 3 0
7 3 0
7
7 3 1
7
7 3 2


In [0]:
allocate_key(3)

In [15]:
print(identifier_circle)
diplay_node_contents(1)
diplay_node_contents(3)
diplay_node_contents(7)
# x = find_successor(identifier_circle[0][1], 2)
# print(x.id)  

[(1, <__main__.Node object at 0x7faf1bc54d30>), (3, <__main__.Node object at 0x7faf1bc54be0>), (7, <__main__.Node object at 0x7faf1bc54eb8>)]
keys stored are.....
1
id is:
1
predecessor is:
finger table is:
index:0 start2 <__main__.Node object at 0x7faf1bc54be0>
index:1 start3 <__main__.Node object at 0x7faf1bc54eb8>
index:2 start5 <__main__.Node object at 0x7faf1bc54eb8>
keys stored are.....
3
id is:
3
predecessor is:
finger table is:
index:0 start4 <__main__.Node object at 0x7faf1bc54eb8>
index:1 start5 <__main__.Node object at 0x7faf1bc54eb8>
index:2 start7 <__main__.Node object at 0x7faf1bc54eb8>
keys stored are.....
7
id is:
7
predecessor is:
finger table is:
index:0 start0 <__main__.Node object at 0x7faf1bc54d30>
index:1 start1 <__main__.Node object at 0x7faf1bc54d30>
index:2 start3 <__main__.Node object at 0x7faf1bc54d30>


In [15]:
find_successor(identifier_circle[1][1],3)

1


In [16]:
join(4)

3
3
6
1
1 4 0
1
1 4 1
6 4 1
6
6 4 2


In [0]:
allocate_key(4)

In [18]:
print(identifier_circle)
diplay_node_contents(1)
diplay_node_contents(3)
diplay_node_contents(4)
diplay_node_contents(6)

[(1, <__main__.Node object at 0x7feef15ef400>), (3, <__main__.Node object at 0x7feef15ef978>), (4, <__main__.Node object at 0x7feef15ef198>), (6, <__main__.Node object at 0x7feef15ef9e8>)]
keys stored are.....
1
id is:
1
predecessor is:
finger table is:
index:0 start2 <__main__.Node object at 0x7feef15ef978>
index:1 start3 <__main__.Node object at 0x7feef15ef198>
index:2 start5 <__main__.Node object at 0x7feef15ef9e8>
keys stored are.....
3
id is:
3
predecessor is:
finger table is:
index:0 start4 <__main__.Node object at 0x7feef15ef9e8>
index:1 start5 <__main__.Node object at 0x7feef15ef9e8>
index:2 start7 <__main__.Node object at 0x7feef15ef400>
keys stored are.....
4
id is:
4
predecessor is:
finger table is:
index:0 start5 <__main__.Node object at 0x7feef15ef9e8>
index:1 start6 <__main__.Node object at 0x7feef15ef9e8>
index:2 start0 <__main__.Node object at 0x7feef15ef400>
keys stored are.....
6
id is:
6
predecessor is:
finger table is:
index:0 start7 <__main__.Node object at 0x7feef

In [21]:
find_successor(identifier_circle[3][1], 2)

1


In [19]:
#calling network stabilization 

# import time 

# while(True):
  
#   for i in range(len(identifier_circle)):
#     stabilize(identifier_circle[i][1])
#     time.sleep(1)

# #calling fix_fingers periodically

# while(True):
  
  for i in range(len(identifier_circle)):
    fix_fingers(identifier_circle[i][1])
#     time.sleep(3)

    

3
6
6
1
